In [6]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import time
import datetime
import numpy as np
import json

In [3]:
start = time.time()

years = list(np.arange(2016,2022,1))
cals = [2] # 1 for WT, 2 for UCI, etc.
months = ['01','02','03','04','05','06','07','08','09','10','11','12']
results_dict = {}
stages_dict = {}

for year in years:
    for month in months:
        for cal in cals:
            url = 'https://firstcycling.com/race.php?y='+str(year)+'&t='+str(cal)+'&m='+str(month)
            r = requests.get(url)
            soup = BeautifulSoup(r.text,'html.parser')

            if year == datetime.datetime.today().year and month == datetime.datetime.today().month: 
                try:
                    tab = soup.find_all('table', class_='sortTabell tablesorter')[1]
                except:
                    continue
            else:
                try:
                    tab = soup.find_all('table', class_='sortTabell tablesorter')[0]
                except:
                    continue

            refs = [i['href'] for i in tab.find_all('a', href=True)]
            races_raw = [x for x in refs if 'race' in x]
            races = [z for z in races_raw if 'k' not in z]
            urls = ['https://firstcycling.com/'+y for y in races]
            for race in urls:
                r_race = requests.get(race)
                soup_race = BeautifulSoup(r_race.text,'html.parser')
                race_name = soup_race.find('h1').text
                rows = soup_race.find_all('tbody')[0].find_all('tr')
                result = []
                for tr in rows:
                    td = tr.find_all('td')
                    row = [tr.text.strip() for tr in td if tr.text.strip()]
                    if row:
                        result.append(row)

                stage_soup = BeautifulSoup(requests.get(race+'&k=2').text,'html.parser') 
                try:
                    if stage_soup.find('h1', class_='delinfo').text == 'Stages':
                        stg_ref = [i['href'] for i in stage_soup.find_all('table', class_="tablesorter")[1].find_all('a')]
                        stages = [x for x in stg_ref if 'race' in x]
                        stages_url = ['https://firstcycling.com/'+y for y in stages]
                        for stage in stages_url:
                            r_stage = requests.get(stage)
                            soup_stage = BeautifulSoup(r_stage.text,'html.parser')
                            for i in soup_stage.find_all('select')[1].find_all('option'):
                                if i.has_attr('selected'):
                                    stage_name = race_name+' stage '+i.text
                            rows_stage = soup_stage.find_all('tbody')[0].find_all('tr')
                            result_stage = []
                            for tr in rows_stage:
                                td = tr.find_all('td')
                                row = [tr.text.strip() for tr in td if tr.text.strip()]
                                if row:
                                    result_stage.append(row)
                            try: 
                                if len(result_stage[0]) == 6:
                                    stages_dict[stage_name] = pd.DataFrame(result_stage,
                                                                   columns=['Position', 'Rider', 'Born', 'Team', 'Total time', 'FC points'])
                                elif len(result_stage[0]) == 5:
                                    stages_dict[stage_name] = pd.DataFrame(result_stage,
                                                                   columns=['Position', 'Rider', 'Born', 'Team', 'Total time'])
                            except:
                                continue
                except:
                    continue


                try:
                    if len(result[0]) == 6:
                        results_dict[race_name] = pd.DataFrame(result, 
                                                           columns=['Position', 'Rider', 'Born', 'Team', 'Total time', 'FC points'])
                    elif len(result[0]) == 5:   
                        results_dict[race_name] = pd.DataFrame(result, 
                                                           columns=['Position', 'Rider', 'Born', 'Team', 'Total time'])
                except ValueError:
                    continue

        print('loop towards month', month, year, 'ended')

end = time.time()
print('elapsed time: '+str((end-start)/60)+' minutes')

loop towards month 01 2016 ended
loop towards month 02 2016 ended
loop towards month 03 2016 ended
loop towards month 04 2016 ended
loop towards month 05 2016 ended
loop towards month 06 2016 ended
loop towards month 07 2016 ended
loop towards month 08 2016 ended
loop towards month 09 2016 ended
loop towards month 10 2016 ended
loop towards month 11 2016 ended
loop towards month 12 2016 ended
loop towards month 01 2017 ended
loop towards month 02 2017 ended
loop towards month 03 2017 ended
loop towards month 04 2017 ended
loop towards month 05 2017 ended
loop towards month 06 2017 ended
loop towards month 07 2017 ended
loop towards month 08 2017 ended
loop towards month 09 2017 ended
loop towards month 10 2017 ended
loop towards month 11 2017 ended
loop towards month 12 2017 ended
loop towards month 01 2018 ended
loop towards month 02 2018 ended
loop towards month 03 2018 ended
loop towards month 04 2018 ended
loop towards month 05 2018 ended
loop towards month 06 2018 ended
loop towar

In [4]:
full_dicts = {**results_dict, **stages_dict}

In [13]:
# convert dataframes into dictionaries
data_dict = {
    key: full_dicts[key].to_dict(orient='records') 
    for key in full_dicts.keys()
}

# write to disk
with open('results.json', 'w') as fp:
    json.dump(
        data_dict, 
        fp, 
        indent=4, 
        sort_keys=True
    )

In [15]:
# read from disk
with open('results.json', 'r') as fp:
    data_dict = json.load(fp)

# convert dictionaries into dataframes
full_dicts = {
    key: pd.DataFrame(data_dict[key]) 
    for key in data_dict
}

In [20]:
final_df = pd.DataFrame()
for key, value in full_dicts.items():
    df = value
    df.loc[:,'race'] = key
    final_df = pd.concat([df, final_df], 0)
final_df

,Born,FC points,Position,Rider,Team,Total time,race
0,1996,14,01,Jannik Steimle,Team Vorarlberg Santic,02:50,Österreich-Rundfahrt - 2019 stage Prologue : W...
1,1989,5,02,Matthias Brändle,Israel Cycling Academy,+ 01,Österreich-Rundfahrt - 2019 stage Prologue : W...
2,1987,3,03,Pieter Vanspeybrouck,Wanty - Groupe Gobert,+ 03,Österreich-Rundfahrt - 2019 stage Prologue : W...
3,1996,None,04,Tom Wirtgen,Wallonie Bruxelles,+ 04,Österreich-Rundfahrt - 2019 stage Prologue : W...
4,1992,None,05,Emils Liepins,Wallonie Bruxelles,+ 04,Österreich-Rundfahrt - 2019 stage Prologue : W...
...,...,...,...,...,...,...,...
137,1982,NaN,DNF,Linus Gerdemann,Stölting Service Group,None,4 Jours de Dunkerque - 2016
138,1992,NaN,DNF,Lasse Norman Hansen,Stölting Service Group,None,4 Jours de Dunkerque - 2016
139,1994,NaN,DNF,Maxime Farazijn,Topsport Vlaanderen,None,4 Jours de Dunkerque - 2016
140,1985,NaN,DNF,Olivier Pardini,Wallonie Bruxelles,None,4 Jours de Dunkerque - 2016


In [26]:
final_df.to_csv('results.csv')

In [25]:
final_df[final_df['race']=='Coppi e Bartali - 2021 stage 01 : Gatteo - Gatteo']

,Born,FC points,Position,Rider,Team,Total time,race
0,1994,14,01,Jakub Mareczko,Vini Zabú,02:19:05,Coppi e Bartali - 2021 stage 01 : Gatteo - Gatteo
1,1985,5,02,Mark Cavendish,Deceuninck-Quick Step,+ 00,Coppi e Bartali - 2021 stage 01 : Gatteo - Gatteo
2,2000,3,03,Marius Mayrhofer,Team DSM,+ 00,Coppi e Bartali - 2021 stage 01 : Gatteo - Gatteo
3,1999,None,04,Luca Coati,Team Qhubeka,+ 00,Coppi e Bartali - 2021 stage 01 : Gatteo - Gatteo
4,1998,None,05,Ethan Hayter,INEOS Grenadiers,+ 00,Coppi e Bartali - 2021 stage 01 : Gatteo - Gatteo
...,...,...,...,...,...,...,...
166,1993,None,167,Mikel Aristi,Euskaltel-Euskadi,+ 00,Coppi e Bartali - 2021 stage 01 : Gatteo - Gatteo
167,1992,None,168,Antonio Angulo,Euskaltel-Euskadi,+ 00,Coppi e Bartali - 2021 stage 01 : Gatteo - Gatteo
168,1994,None,DNS,Gianni Moscon,INEOS Grenadiers,None,Coppi e Bartali - 2021 stage 01 : Gatteo - Gatteo
169,1996,None,DNF,Ibai Azurmendi,Euskaltel-Euskadi,None,Coppi e Bartali - 2021 stage 01 : Gatteo - Gatteo


#Conversione secondi

La cella che segue converte in secondi i tempi delle gare, calcola i distacchi dal primo e i tempi in percentuale per tutto il df di risultati. Va ancora resa come funzione.

In [52]:
race_list = final_df['race'].unique()                   ## generates list of single races given the df
df = pd.DataFrame(columns=['Position', 'Rider', 'Born', 'Team', 'Total time', 'FC points', 'race', 'Time seconds', 'Perc of time']) ## empty df to copy results

for race in race_list:
    df_temp = final_df.loc[final_df['race'] == race]
    temp = df_temp['Total time']
    temp_list = []
    for x in temp:
        if x is not None:
            y = x.split(':')                            ## split it from HH:MM:SS to ['HH', 'MM', 'SS'] (it is a list of strings)
            for i in range(0, len(y)):
                y[i]=int(y[i].replace("+ ", ""))        ## deletes '+ ' from the time gaps
            temp_int = 0
            for j in range(len(y)-1, -1, -1):
                temp_int += y[j]*(60 **(len(y)-(j+1)))  ## converts timegaps into seconds 
            temp_list.append(temp_int)                  ## appends them to list

    for k in range(len(temp_list), len(temp)):
        temp_list.append('None')                        ## appends 'None' to list before merging into DF
    df_temp['Time seconds'] = temp_list

    df_temp['Perc of time'] = ""  ## creates empty column for percentage of time

    for index in range(1, len(df_temp['Time seconds'])):      ## sums timegaps and total time of the winner
        if df_temp['Time seconds'][index] != 'None':
            df_temp['Time seconds'][index] +=  int(df_temp['Time seconds'][0])

    for index  in range(len(df_temp['Time seconds'])) :
        if df_temp['Time seconds'][index] != 'None':
            df_temp['Perc of time'][index] = int(df_temp['Time seconds'][index]) / int(df_temp['Time seconds'][0])
        else:
            df_temp['Perc of time'][index] = 'None'
    df = pd.concat([df,df_temp])

final_df = df                                         
del df                                                  ## deletes df


<ipython-input-52-0d50c37d37ac>:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_temp['Time seconds'] = temp_list
<ipython-input-52-0d50c37d37ac>:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_temp['Perc of time'] = ""  ## creates empty column for percentage of time
/Users/giacomolini/Library/Python/3.9/lib/python/site-packages/IPython/core/interactiveshell.py:3437: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: htt

https://firstcycling.com/race.php?y=2020&t=2&m=01
https://firstcycling.com/race.php?y=2020&t=2&m=02
https://firstcycling.com/race.php?y=2020&t=2&m=03
https://firstcycling.com/race.php?y=2020&t=2&m=04
https://firstcycling.com/race.php?y=2020&t=2&m=05
https://firstcycling.com/race.php?y=2020&t=2&m=06
https://firstcycling.com/race.php?y=2020&t=2&m=07
https://firstcycling.com/race.php?y=2020&t=2&m=08
https://firstcycling.com/race.php?y=2020&t=2&m=09
https://firstcycling.com/race.php?y=2020&t=2&m=10
https://firstcycling.com/race.php?y=2020&t=2&m=11
https://firstcycling.com/race.php?y=2020&t=2&m=12
